# Install required packages

In [ ]:
! pip install lm_eval  langdetect -q
! pip install git+https://github.com/felipemaiapolo/tinyBenchmarks

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 64.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 9.5 MB/s eta 0:00:00
  Cloning https://github.com/felipemaiapolo/tinyBenchmarks to /tmp/pip-req-build-bpckm2tu
  Running command git clone 

# List all available benchmarks

In [ ]:
! lm_eval --tasks list

Streaming output truncated to the last 5000 lines.
|global_mmlu_full_sw_professional_accounting                                           |lm_eval/tasks/global_mmlu/full/sw/global_mmlu_full_sw_professional_accounting.yaml                                                                              |multiple_choice      |
|global_mmlu_full_sw_professional_law                                                  |lm_eval/tasks/global_mmlu/full/sw/global_mmlu_full_sw_professional_law.yaml                                                                                     |multiple_choice      |
|global_mmlu_full_sw_professional_medicine                                             |lm_eval/tasks/global_mmlu/full/sw/global_mmlu_full_sw_professional_medicine.yaml                                                                                |multiple_choice      |
|global_mmlu_full_sw_professional_psychology                                           |lm_eval/tasks/global_mmlu/full/sw/global_mmlu

# Run a benchmark

In [ ]:
import torch
from lm_eval import evaluator
from joblib import dump
from huggingface_hub import login
from google.colab import userdata


HF_TOKEN = userdata.get('HF_TOKEN')

login(HF_TOKEN)

config = {
    "model": "meta-llama/Llama-3.2-1B-Instruct",
    "tasks": ["tinyGSM8k"],
    "batch_size": "2",
}

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

results = evaluator.simple_evaluate(
    model="hf",
    model_args=f"pretrained={config['model']},parallelize=True,trust_remote_code=True",
    tasks=config["tasks"],
    device=device,
    batch_size=config["batch_size"],
)


dump(results, "evaluation_results.joblib")


        chat template is not applied. Recommend setting `apply_chat_template` (optionally `fewshot_as_multiturn`).
`torch_dtype` is deprecated! Use `dtype` instead!
Running generate_until requests: 100%|██████████| 100/100 [09:58<00:00,  5.99s/it]


['evaluation_results.joblib']

In [ ]:
results.keys()

dict_keys(['results', 'group_subtasks', 'configs', 'versions', 'n-shot', 'higher_is_better', 'n-samples', 'samples', 'config', 'git_hash', 'date', 'pretty_env_info', 'transformers_version', 'lm_eval_version', 'upper_git_hash', 'tokenizer_pad_token', 'tokenizer_eos_token', 'tokenizer_bos_token', 'eot_token_id', 'max_length'])

In [ ]:
results['results']

{'tinyGSM8k': {'alias': 'tinyGSM8k',
  'exact_match,strict-match': np.float64(0.3901494861944388),
  'exact_match_stderr,strict-match': 'N/A',
  'exact_match,flexible-extract': np.float64(0.3901494861944388),
  'exact_match_stderr,flexible-extract': 'N/A'}}

In [ ]:
results['samples']['tinyGSM8k'][0].keys()

dict_keys(['doc_id', 'doc', 'target', 'arguments', 'resps', 'filtered_resps', 'filter', 'metrics', 'doc_hash', 'prompt_hash', 'target_hash', 'exact_match'])

# Examine samples manually

In [ ]:
def print_sample(i):
  print(f"""
  Question: {results['samples']['tinyGSM8k'][i]['doc']['question']}\n\n
  Target: {results['samples']['tinyGSM8k'][i]['target']}\n\n
  Answer: {results['samples']['tinyGSM8k'][i]['resps'][0][0]}
  """)

In [ ]:
print_sample(0)


  Question: Rory orders 2 subs for $7.50 each, 2 bags of chips for $1.50 each and 2 cookies for $1.00 each for delivery.  There’s a 20% delivery fee added at check out and she wants to add a $5.00 tip.  What will her delivery order cost?


  Target: 2 subs are $7.50 each so that’s 2*7.50 = $<<2*7.5=15.00>>15.00
2 bags of chips are $1.50 each so that’s 2*1.50 = $<<2*1.50=3.00>>3.00
2 cookies are $1.00 each so that’s 2*1 = $<<2*1=2.00>>2.00
Her delivery order will be 15+3+2= $<<15+3+2=20.00>>20.00
There’s a 20% delivery fee on the $20.00 which adds .20*20 = $4.00 to her bill
The delivery order is $20.00, there’s a $4.00 delivery fee and she adds a $5.00 tip for a total of 20+4+5 = $<<20+4+5=29.00>>29.00
#### 29


  Answer:  Rory will spend $7.50 x 2 = $15.00 on subs.
She will spend $1.50 x 2 = $3.00 on chips.
She will spend $1.00 x 2 = $2.00 on cookies.
So, the total cost of the subs, chips, and cookies is $15.00 + $3.00 + $2.00 = $20.00.
The delivery fee is 20% of $20.00, which is 0.20

In [ ]:
print_sample(55)


  Question: Miguel uses 2 pads of paper a week for his drawing. If there are 30 sheets of paper on a pad of paper, how many sheets of paper does he use every month?


  Target: Miguel uses 30 x 2 = <<30*2=60>>60 sheets of paper every week.
Therefore, he uses 60 x 4 = <<60*4=240>>240 sheets of paper every month.
#### 240


  Answer:  Miguel uses 2 pads a week, so 2 pads/week * 52 weeks/year = 104 pads/year.
104 pads/year * 12 months/year = 1,248 sheets/year.
1,248 sheets/year * 12 months/year = 14,784 sheets/year.
#### 14,784


  


# Verify the accuracy

In [ ]:
import re

def extract_final_answer(text: str):
    match = re.search(r"####\s*(-?[0-9.,]+)", text)
    if match:
        return match.group(1)
    return None


In [ ]:
correct = 0
incorrect = 0
for i in results['samples']['tinyGSM8k']:
  target = i['target']
  answer = i['resps'][0][0]

  final_answer = extract_final_answer(answer)
  final_target = extract_final_answer(target)

  if final_answer == final_target:
    correct += 1
  else:
    incorrect += 1


In [ ]:
correct / (correct + incorrect)

0.39

In [ ]:
correct

78

In [ ]:
incorrect

122